<a href="https://colab.research.google.com/github/chambai/Deep_Learning_Course/blob/main/Week%201%20DL%202/HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb
## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*. 

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for i.e. the Adam optimiser.

In this tutorial, you will use the Keras Tuner to perform hypertuning for the MNIST handwritten digit image dataset with a feed-forward neural network.
It will try the number of units from 32 to 512 in the first hidden layer and try
learning rates of 0.01, 0.001, and 0.0001.

In [ ]:
# install the Keras-tuner library
!pip install keras-tuner

In [ ]:
# import the library and refer to it as kt
import kerastuner as kt

MNIST dataset of handwritten digits has images of digits from 0 to 9, has a training set of 60000 and a test set of 10000

In [ ]:
# load the mnist dataset from keras
import tensorflow as tf
from tensorflow.keras import datasets
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

Investigate the shape of the dataset

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Each pixel of the picture is represented as a number between 0 and 255. Therefore we can normalize this data by dividing by 255 to ensure all input values to the deep neural network will be between 0 and 1 

In [ ]:
# Normalize pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# apply one-hot-encoding to the output data
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

The model is built the same as we have done so far, using keras.Sequential().  The only difference is that this code is now within the function `def model_builder(hp)`.

There is an extra step of `hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)` specifying the range by which the numner of units in the first layer will be changed by.

There is an extra step of `hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])` specifying what the learning rate of the optimiser is to be varied by.  This is then passed to the optimiser as the learning_rate parameter

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

# build the model
def model_builder(hp):
  model = tf.keras.Sequential()
  # specify the input layer separately to the first layer
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Specify a change in the number of units in this layer 
  # from 32 to 512 in steps of 32
  # We have specified the name of this hyperparameter as 'hidden_layer_1_units'
  # this name will appear in the output text when running tuner.search function later on
  hp_units = hp.Int('hidden_layer_1_units', min_value = 32, max_value = 512, step = 32)
  # specify the range of units (hp_units) as the units input parameter to this layer of the model
  model.add(Dense(units=hp_units, activation='relu'))

  # Specify the output layer
  model.add(Dense(units=10, activation='softmax'))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  # create the Adam optimiser, specifying the variable learning rate
  opt = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate)
  # set the optimiser in the compile method
  model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
  
  return model

Setup the tuner by calling `kt.Hyperband`  and specifying the model_builder function as the first parameter

The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket using the `factor` and `max_epochs` parameters.  A bracket is a tree diagram that represents the series of games played during a sports knockout tournament)

In [ ]:
# instatiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,            # factor is a number that determines how many models are created to run in parallel whaen testing the hyperparameters
                     directory = 'my_dir', # directory that the tuned hyperparameter results are stored in (change the name of this directory if you get the message INFO:tensorflow:Oracle triggered exit)
                     project_name = 'intro_to_kt') 

A Callback is defined to clear the screen during the search for the hyperparameters.  A callback is just a way for the tuner to signal that it has reached the end of training for each hyperparameter setup.  The callback code below then clears the screen.

In [ ]:
# define a callback to clear the training outputs at the end of every training step
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

The hyperparameter search is performed using `tuner.search`.  The arguments are the same as for the fit method for the model.
When the search is complete, it returns the best hyperparameters.

**Note:** if you do not get a display of updating training epochs when running the next cell and you only see this message:   **INFO:tensorflow:Oracle triggered exit**, the keras tuner has not worked correctly.  To fix this, 
update `directory='my_dir'` (which is two code cells above) to have a different directory name. Keras stores the tuning results in this directory and can sometimes error if you are changing the model alot.

In [ ]:
import IPython
# run the hyperparameter search
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test), callbacks = [ClearTrainingOutput()])

The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

In [ ]:
# print out the hyperparameters keras tuner has determined to be the best values
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

Now we can train the model with the optimum parameters found in the previous step which are stored in `best_hps`

In [ ]:
# set the model to have the optimum hyperparameters and print the summary so we can see the 
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
# train the model as usual
history = model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Plot the accuracy and loss of the model

In [ ]:
# plot the history of the training
import matplotlib.pyplot as plt

# summarize the history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

# summarize the history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()